## Ran a bunch of queries to examine encounter types, and count the frequencies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime
from impala.dbapi import connect
from impala.util import as_pandas
%matplotlib notebook
plt.style.use('ggplot')

In [ ]:
conn = connect(host="mycluster.domain.com", port=my_impala_port_number)
cur = conn.cursor()
cur.execute("use my_db")

In [ ]:
# Get the admit type description and counts for RRT encounters

query_admit_RRT = """
SELECT 
      enc.admit_type_cd
    , cv_admit_type.description AS admit_type_des
    , count(1)
FROM encounter enc 
LEFT OUTER JOIN code_value cv_admit_type 
ON enc.admit_type_cd = cv_admit_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type 
ON enc.encntr_type_cd = cv_enc_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type_class 
ON enc.encntr_type_class_cd = cv_enc_type_class.code_value 
WHERE enc.encntr_id IN (
                        SELECT encntr_id 
                        FROM clinical_event 
                        WHERE event_cd = '54411998' 
                        AND result_status_cd NOT IN ('31', '36') 
                        AND valid_until_dt_tm > 4000000000000 
                        AND event_class_cd not in ('654645')
                        ) 
AND (enc.encntr_complete_dt_tm < 4000000000000) 
AND enc.loc_facility_cd = '633867' 
GROUP BY enc.admit_type_cd, admit_type_des;
"""
cur.execute(query_admit_RRT)
cur.fetchall()

In [ ]:
# Get the distribution for non-RRT encounters over the reasons for the admission type description

query_admit_NotRRT = """
SELECT 
      enc.admit_type_cd
    , cv_admit_type.description AS admit_type_des
    , count(1)
FROM encounter enc 
LEFT OUTER JOIN code_value cv_admit_type 
ON enc.admit_type_cd = cv_admit_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type 
ON enc.encntr_type_cd = cv_enc_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type_class 
ON enc.encntr_type_class_cd = cv_enc_type_class.code_value 
WHERE enc.encntr_id NOT IN (
                            SELECT encntr_id 
                            FROM clinical_event 
                            WHERE event_cd = '54411998' 
                            AND result_status_cd NOT IN ('31', '36') 
                            AND valid_until_dt_tm > 4000000000000 
                            AND event_class_cd not in ('654645')
                            ) 
AND (enc.encntr_complete_dt_tm < 4000000000000) 
AND enc.loc_facility_cd = '633867' 
GROUP BY enc.admit_type_cd, admit_type_des;
"""
cur.execute(query_admit_NotRRT)
cur.fetchall()

In [ ]:
# Count the number of non-RRT encounters

query_count_NotRRT = """
SELECT count(1) 
FROM (
        SELECT enc.admit_type_cd 
        FROM encounter enc 
        LEFT OUTER JOIN code_value cv_admit_type 
        ON enc.admit_type_cd = cv_admit_type.code_value 
        LEFT OUTER JOIN code_value cv_enc_type 
        ON enc.encntr_type_cd = cv_enc_type.code_value 
        LEFT OUTER JOIN code_value cv_enc_type_class 
        ON enc.encntr_type_class_cd = cv_enc_type_class.code_value 
        WHERE enc.encntr_id NOT IN (
                                    SELECT encntr_id 
                                    FROM clinical_event 
                                    WHERE event_cd = '54411998' 
                                    AND result_status_cd NOT IN ('31', '36') 
                                    AND valid_until_dt_tm > 4000000000000 
                                    AND event_class_cd not in ('654645')
                                    ) 
AND (enc.encntr_complete_dt_tm < 4000000000000) 
AND enc.loc_facility_cd = '633867') t;
"""
cur.execute(query_count_NotRRT)
cur.fetchall()

In [ ]:
# Check the distribution for RRT encoutners over the encounter 
# types of 'Inpatient', 'Outpatient' and 'Observation'

query_enctypecl_RRT = """
SELECT 
      enc.encntr_type_class_cd
    , cv_enc_type_class.description AS enc_type_class_desc
    , count(1)
FROM encounter enc 
LEFT OUTER JOIN code_value cv_admit_type 
ON enc.admit_type_cd = cv_admit_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type 
ON enc.encntr_type_cd = cv_enc_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type_class 
ON enc.encntr_type_class_cd = cv_enc_type_class.code_value 
WHERE enc.encntr_id IN (
                        SELECT encntr_id 
                        FROM clinical_event 
                        WHERE event_cd = '54411998' 
                        AND result_status_cd NOT IN ('31', '36') 
                        AND valid_until_dt_tm > 4000000000000 
                        AND event_class_cd not in ('654645')
                        ) 
AND (enc.encntr_complete_dt_tm < 4000000000000) 
AND enc.loc_facility_cd = '633867' 
GROUP BY enc.encntr_type_class_cd, enc_type_class_desc;
"""
cur.execute(query_enctypecl_RRT)
cur.fetchall()

In [ ]:
# Check the distribution for RRT encounters over the admit types 
# of 'Urgent', 'Emergency', 'None', and 'Elective'

query_enctypecl_RRT_admit = """
SELECT 
      enc.admit_type_cd
    , cv_admit_type.description AS admit_type_des
    , count(1)
FROM encounter enc 
LEFT OUTER JOIN code_value cv_admit_type 
ON enc.admit_type_cd = cv_admit_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type 
ON enc.encntr_type_cd = cv_enc_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type_class 
ON enc.encntr_type_class_cd = cv_enc_type_class.code_value 
WHERE enc.encntr_id IN (
                        SELECT encntr_id 
                        FROM clinical_event 
                        WHERE event_cd = '54411998' 
                        AND result_status_cd NOT IN ('31', '36') 
                        AND valid_until_dt_tm > 4000000000000 
                        AND event_class_cd not in ('654645')
                        ) 
AND (enc.encntr_complete_dt_tm < 4000000000000) 
AND enc.loc_facility_cd = '633867' 
AND enc.encntr_type_class_cd = '391'
GROUP BY enc.admit_type_cd, admit_type_des;
"""
cur.execute(query_enctypecl_RRT_admit)
cur.fetchall()

In [ ]:
# Check the distribution for non-RRT encounters over the encounter types 

query_enctype_NotRRT = """
SELECT 
      enc.encntr_type_class_cd
    , cv_enc_type_class.description AS enc_type_class_desc
    , count(1)
FROM encounter enc 
LEFT OUTER JOIN code_value cv_admit_type 
ON enc.admit_type_cd = cv_admit_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type 
ON enc.encntr_type_cd = cv_enc_type.code_value 
LEFT OUTER JOIN code_value cv_enc_type_class 
ON enc.encntr_type_class_cd = cv_enc_type_class.code_value 
WHERE enc.encntr_id NOT IN (
                            SELECT encntr_id 
                            FROM clinical_event 
                            WHERE event_cd = '54411998' 
                            AND result_status_cd NOT IN ('31', '36') 
                            AND valid_until_dt_tm > 4000000000000 
                            AND event_class_cd not in ('654645')
                            ) 
AND (enc.encntr_complete_dt_tm < 4000000000000) 
AND enc.loc_facility_cd = '633867' 
GROUP BY enc.encntr_type_class_cd, enc_type_class_desc;
"""
cur.execute(query_enctype_NotRRT)
cur.fetchall()